<a href="https://colab.research.google.com/github/Makerov/Adult_Income_Raiff/blob/main/income_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключение к БД

In [ ]:
!mkdir -p ~/.postgresql && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
    --output-document ~/.postgresql/root.crt && \
chmod 0600 ~/.postgresql/root.crt
!sudo apt update && sudo apt install --yes python3 python3-pip && \
!pip3 install psycopg2-binary

--2023-02-20 11:19:54--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/root/.postgresql/root.crt’

/root/.postgresql/r 100%[===================>]   3.50K  --.-KB/s    in 0s      

2023-02-20 11:19:55 (654 MB/s) - ‘/root/.postgresql/root.crt’ saved [3579/3579]

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:6 https://developer.downl

Подключение библиотек и выгрузка датасета

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

conn = psycopg2.connect("""
    host=rc1b-70q90cunwkzx49mj.mdb.yandexcloud.net
    port=6432
    sslmode=verify-full
    dbname=db_clients
    user=mlhs_student
    password=mlhs_student
    target_session_attrs=read-write
""")

query = "SELECT * from public.adult_income"
data = pd.read_sql_query(query, conn)

conn.close()

data.head(9)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,30.0,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0.0,0.0,40.0,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103.0,0.0,32.0,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K


In [ ]:
data.shape

(9615, 15)

In [ ]:
data.dtypes

age                  int64
workclass           object
fnlwgt               int64
education           object
educational-num      int64
marital-status      object
occupation          object
relationship        object
race                object
gender              object
capital-gain       float64
capital-loss       float64
hours-per-week     float64
native-country      object
income              object
dtype: object

## Сохранение данных

In [ ]:
data.to_csv("clients_class.csv", index=False)

### Разведочный анализ данных

## 1. Оценка качества данных
* пропущенные значения
* ошибочные значения

In [ ]:
len(data)

9615

##### Посмотрим, есть ли в данных пропущенные значения.

In [ ]:
data.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               1
gender             1
capital-gain       1
capital-loss       1
hours-per-week     1
native-country     1
income             1
dtype: int64

In [ ]:
data = data.dropna(subset=['income'])

Заполним пропуски в столбцах: race, gender, capital-gain, capital-loss, hours-per-week, native-country, income

In [ ]:
# data['gender']
# data['income'].value_counts()
# median_cap_g = data['capital-gain'].median
# median_cap_l = data['capital-loss'].median
# median_hou_w = data['hours-per-week'].median

data["race"].fillna('Another', inplace=True)
data["gender"].fillna('Another', inplace=True)
data["capital-gain"].fillna(data['capital-gain'].median, inplace=True)
data["capital-loss"].fillna(data['capital-loss'].median, inplace=True)
data["hours-per-week"].fillna(data['hours-per-week'].median, inplace=True)
data["native-country"].fillna('United-States', inplace=True)
data["income"].fillna('<=50K', inplace=True)

# data.isna().sum()
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [ ]:
data.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

In [ ]:
data.dtypes

age                  int64
workclass           object
fnlwgt               int64
education           object
educational-num      int64
marital-status      object
occupation          object
relationship        object
race                object
gender              object
capital-gain       float64
capital-loss       float64
hours-per-week     float64
native-country      object
income               int64
dtype: object

In [ ]:
data['capital-gain'].value_counts()

0.0        8826
15024.0      99
7688.0       71
7298.0       69
99999.0      55
           ... 
3471.0        1
1264.0        1
25236.0       1
1471.0        1
7443.0        1
Name: capital-gain, Length: 104, dtype: int64

In [ ]:
data['income'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

In [ ]:
"""
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

categorical = ['workclass','marital-status','occupation','relationship', 'race', 'gender']
numeric_features = data.columns

column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('scaling', StandardScaler(), numeric_features)
])

### Основные числовые характеристики

In [ ]:
# data['income'].value_counts()
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
count,9614.000000,9.614000e+03,9614.000000,9614.000000,9614.000000,9614.000000,9614.000000
mean,38.731329,1.882948e+05,10.080092,1121.998232,87.926669,40.522987,0.234866
std,13.842687,1.051352e+05,2.562733,7890.578253,404.085647,12.416221,0.423937
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.154652e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.778450e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.374175e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000,1.000000


In [ ]:
data.describe(include='object') # включение категориальных колонок

,workclass,education,marital-status,occupation,relationship,race,gender,native-country
count,9614,9614,9614,9614,9614,9614,9614,9614
unique,9,16,7,15,6,5,2,41
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States
freq,6573,3121,4307,1222,3800,8255,6409,8681


In [ ]:
corr = data.corr() # построение матрицы корреляций

corr.style.background_gradient(cmap='coolwarm')

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
age,1.000000,-0.081938,0.006905,0.082360,0.052424,0.081728,0.224250
fnlwgt,-0.081938,1.000000,-0.024398,-0.005989,-0.000409,-0.004686,-0.003089
educational-num,0.006905,-0.024398,1.000000,0.132639,0.083546,0.136237,0.320775
capital-gain,0.082360,-0.005989,0.132639,1.000000,-0.030944,0.098911,0.221998
capital-loss,0.052424,-0.000409,0.083546,-0.030944,1.000000,0.056343,0.139776
hours-per-week,0.081728,-0.004686,0.136237,0.098911,0.056343,1.000000,0.228856
income,0.224250,-0.003089,0.320775,0.221998,0.139776,0.228856,1.000000


In [ ]:
data = data.drop(['fnlwgt'],axis=1)

In [ ]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,age,educational-num,capital-gain,capital-loss,hours-per-week,income
age,1.000000,0.006905,0.082360,0.052424,0.081728,0.224250
educational-num,0.006905,1.000000,0.132639,0.083546,0.136237,0.320775
capital-gain,0.082360,0.132639,1.000000,-0.030944,0.098911,0.221998
capital-loss,0.052424,0.083546,-0.030944,1.000000,0.056343,0.139776
hours-per-week,0.081728,0.136237,0.098911,0.056343,1.000000,0.228856
income,0.224250,0.320775,0.221998,0.139776,0.228856,1.000000


In [ ]:
X = data.drop('income', axis=1)
y = data['income']

In [ ]:
#X = X.drop(['workclass','marital-status','education', 'occupation','relationship', 'race', 'gender'], axis=1)
X = X.drop(['native-country'], axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(Xtrain)

Xtrain = pd.DataFrame(ss.transform(Xtrain), columns=X.columns)
Xtest = pd.DataFrame(ss.transform(Xtest), columns=X.columns)

Xtrain.head()

,age,educational-num,capital-gain,capital-loss,hours-per-week
0,-1.436662,-0.032251,-0.145108,-0.221089,-2.160013
1,-1.291984,-0.032251,-0.145108,-0.221089,-0.043911
2,-1.002627,-0.419154,-0.145108,-0.221089,0.769974
3,-0.568591,1.128459,-0.145108,-0.221089,-0.450854
4,-0.423912,1.128459,-0.145108,-0.221089,1.176916


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression() # объявление модели
model.fit(Xtrain, ytrain) # обучение модели

LogisticRegression()

In [ ]:
pred_test = model.predict(Xtest) # предсказание классов

In [ ]:
accuracy_score(ytest, pred_test)

0.8157237936772047